In [2]:
# ------------------------------------------------------------
# Summer EVI median (single year), Landsat 8 SR L2, 30 m
# Per-state exports — same structure as the NDVI script
# ------------------------------------------------------------

import ee
import pandas as pd
from pathlib import Path

# ------------------------------------------------------------
# 0) Local paths
# ------------------------------------------------------------
REPO_ROOT = Path(r"C:\Users\bdevoe\Desktop\Greenspace")
STATE_FIPS_FILE = REPO_ROOT / "resources" / "StateFipsUsps.csv"
DRIVE_FOLDER = "EVI_DOWNLOAD_2021"

# ------------------------------------------------------------
# 1) Parameters
# ------------------------------------------------------------
YEAR = 2021
SUMMER_START = "06-01"
SUMMER_END   = "09-01"

DATASET_ID = "LANDSAT/LC08/C02/T1_L2"
SCALE_M = 30

SR_SCALE  = 0.0000275
SR_OFFSET = -0.2

# ------------------------------------------------------------
# 2) EE Init
# ------------------------------------------------------------
ee.Initialize()

# ------------------------------------------------------------
# 3) State list
# ------------------------------------------------------------
state_map_df = pd.read_csv(STATE_FIPS_FILE, dtype=str)
STATES = state_map_df["USPS"].str.upper().tolist()

EXCLUDE = {"AK", "HI", "PR", "VI", "GU", "AS", "MP"}
STATES = [s for s in STATES if s not in EXCLUDE]

print("Exporting states:", STATES)

# ------------------------------------------------------------
# 4) State geometries
# ------------------------------------------------------------
states_fc = (
    ee.FeatureCollection("TIGER/2018/States")
    .filter(ee.Filter.inList("STUSPS", STATES))
)

# ------------------------------------------------------------
# 5) Build EVI summer composite
# ------------------------------------------------------------
start = f"{YEAR}-{SUMMER_START}"
end   = f"{YEAR}-{SUMMER_END}"

ic = (
    ee.ImageCollection(DATASET_ID)
    .filterDate(start, end)
)

def mask_cloud_shadow(img):
    qa = img.select("QA_PIXEL")
    cloud  = qa.bitwiseAnd(1 << 3).eq(0)
    shadow = qa.bitwiseAnd(1 << 4).eq(0)
    return img.updateMask(cloud.And(shadow))

ic = ic.map(mask_cloud_shadow)

def scale_sr(img):
    scaled = img.select("SR_B.*").multiply(SR_SCALE).add(SR_OFFSET)
    return img.addBands(scaled, overwrite=True)

ic = ic.map(scale_sr)

# EVI formula
def add_evi(img):
    evi = img.expression(
        "2.5 * ((NIR - RED) / (NIR + 6*RED - 7.5*BLUE + 1))",
        {
            "NIR":  img.select("SR_B5"),
            "RED":  img.select("SR_B4"),
            "BLUE": img.select("SR_B2")
        }
    ).rename("EVI")
    return img.addBands(evi)

ic = ic.map(add_evi)

# Median EVI in float space
evi_summer_float = ic.select("EVI").median()

# Scale to int16: clamp to [-1, 1], multiply by 10000, cast to int16
evi_summer = (
    evi_summer_float
    .clamp(-1, 1)
    .multiply(10000)
    .toInt16()
    .rename("EVI")
)

# ------------------------------------------------------------
# 6) Per-state Drive exports
# ------------------------------------------------------------
for st in STATES:
    geom = states_fc.filter(ee.Filter.eq("STUSPS", st)).geometry()
    filename = f"{st}_{YEAR}_summer_evi_{SCALE_M}m"

    print(f"Queueing export: {filename}")

    ee.batch.Export.image.toDrive(
        image           = evi_summer.clip(geom),
        description     = filename,
        folder          = DRIVE_FOLDER,
        fileNamePrefix  = filename,
        region          = geom,
        scale           = SCALE_M,
        maxPixels       = 1e13,
        fileFormat      = "GeoTIFF",
        formatOptions   = {"cloudOptimized": True},
    ).start()

print(f"Exports submitted to Drive/{DRIVE_FOLDER}.")


Exporting states: ['AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
Queueing export: AL_2021_summer_evi_30m
Queueing export: AZ_2021_summer_evi_30m
Queueing export: AR_2021_summer_evi_30m
Queueing export: CA_2021_summer_evi_30m
Queueing export: CO_2021_summer_evi_30m
Queueing export: CT_2021_summer_evi_30m
Queueing export: DE_2021_summer_evi_30m
Queueing export: DC_2021_summer_evi_30m
Queueing export: FL_2021_summer_evi_30m
Queueing export: GA_2021_summer_evi_30m
Queueing export: ID_2021_summer_evi_30m
Queueing export: IL_2021_summer_evi_30m
Queueing export: IN_2021_summer_evi_30m
Queueing export: IA_2021_summer_evi_30m
Queueing export: KS_2021_summer_evi_30m
Queueing export: KY_2021_summer_evi_30m
Queueing export: LA_2021_summer_evi_30m
Queuein